<a href="https://colab.research.google.com/github/Yolantele/ML-data-clasifier/blob/master/tabular_neural_net_classification_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tabular Model For Waste Data Classification


In [0]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [0]:
from fastai import *
from fastai.tabular import *

Tabular data should be in a Pandas `DataFrame`.
df - data_frame

In [181]:
# mount to google drive and load files
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [182]:
# check file is loaded OK
path = '/content/drive/My Drive/data/'
print(path)

/content/drive/My Drive/data/


In [0]:
# load train data frame
df = pd.read_csv(path + '/minimizedData.csv')
df.head()

# load test data frame with material field empty
test_df = pd.read_csv(path + '/testNoMaterialData.csv')

#cats and conts 

**categorical variables - cat_names:**

what you're using to make predictions with.  (description, euralDescription).
for categorical variables - will use Embeddings 

**continuos variables - cont_names:**

any catgeory that has limited number of possible (integer) choises: true/false ; fields such as cleanOrContaminated or mixedOrPure
continuos varibale names - they are sent to Neural net as pixels 

**uses processor - procs:**

a number of processes that deal with data in some ways: FillMissing, Categorify, Normalize both train and test dtata sets

In [0]:
# dependable variable field:
dep_var = 'material'
 
# can add composite1, mType
cat_names = ['euralDescription', 'description', 'composite1', 'mType']
cont_names = ['mixedOrPure', 'cleanOrDirty']
# processors:
# procs = [FillMissing, Categorify, Normalize]

In [0]:
test = TabularList.from_df(df.iloc[800:1000].copy(), path=path + '/minimizedData.csv', cat_names=cat_names, cont_names=cont_names)

Next:
data block api
add labels - tell which cols - column it is 

and get back data bunch
split train and test data ranges 800,1000


In [186]:
data = (TabularList.from_df(df, path=path+ '/minimizedData.csv', cat_names=cat_names, cont_names=cont_names, procs=procs)
                           .split_by_idx(list(range(800,1000)))
                           .label_from_df(cols=dep_var)
                           .add_test(test)
                           .databunch())

/usr/local/lib/python3.6/dist-packages/fastai/data_block.py:537: UserWarning: You are labelling your items with CategoryList.
Your valid set contained the following unknown labels, the corresponding items have been discarded.
zinc, flint, non-iron
  if getattr(ds, 'warn', False): warn(ds.warn)


In [187]:
data.show_batch(rows=10)

euralDescription,description,composite1,mType,mixedOrPure_na,cleanOrDirty_na,mixedOrPure,cleanOrDirty,target
plastics,PET bottles preconsumer - fur,#na#,#na#,False,True,-1.7785,0.2535,PET
soil and stones not covered by 17 05 03,Grond de Klip 16 Koedijk,#na#,#na#,False,True,0.5620,0.2535,ground
soil and stones,"Land, Veenrug 68, Wognum",#na#,#na#,False,True,0.5620,0.2535,ground
soil and stones not covered by 17 05 03,clay / sand contaminated,#na#,"clay soil, sandy soil",False,False,0.5620,0.2535,ground
Glass,Glass companies Apeldoorn,#na#,#na#,True,True,0.5620,0.2535,Glass
wastes not otherwise specified,removal and processing bleaching earth,bleaching earth,#na#,False,True,0.5620,0.2535,bentonite
metal,OLD GRASS IRON,#na#,#na#,True,True,0.5620,0.2535,iron
material unsuitable for consumption or processing,Sunflower fringe technical,dry,sunflower,False,True,0.5620,0.2535,organic material
"mixed construction and demolition wastes other than those mentioned in 17 09 01, 17 09 02 and 17 09 03",clean gravel,#na#,#na#,False,False,-1.7785,-3.9425,grind
soil and stones not covered by 17 05 03,land Living via report <100m3,#na#,#na#,False,True,0.5620,0.2535,ground


pass in data to tabula)learner:


In [0]:
learn = tabular_learner(data, layers=[200,100], metrics=accuracy)

In [189]:
learn.fit(2, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,2.281794,2.101243,0.472081,00:00
1,1.374172,1.083628,0.756345,00:00


In [190]:
# see into the model
learn.model.eval()

TabularModel(
  (embeds): ModuleList(
    (0): Embedding(59, 16)
    (1): Embedding(1774, 106)
    (2): Embedding(39, 12)
    (3): Embedding(159, 27)
    (4): Embedding(3, 3)
    (5): Embedding(3, 3)
  )
  (emb_drop): Dropout(p=0.0, inplace=False)
  (bn_cont): BatchNorm1d(2, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (layers): Sequential(
    (0): Linear(in_features=169, out_features=200, bias=True)
    (1): ReLU(inplace=True)
    (2): BatchNorm1d(200, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): Linear(in_features=200, out_features=100, bias=True)
    (4): ReLU(inplace=True)
    (5): BatchNorm1d(100, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): Linear(in_features=100, out_features=98, bias=True)
  )
)

In [0]:
learn.model_dir = '/content/drive/My Drive/data/models'
learn.save('material_classifier_model3')

## Inference

data_frame (df) 

In [222]:
row = 960
# print('given material -->', df.iloc[row].material)
# print(df.iloc[row])

print('test material -->', test_df.iloc[row].material)
print(test_df.iloc[row])


test material --> nan
reason                                                            NaN
origin                                                            NaN
color                                                             NaN
state                                                             NaN
size                                                              NaN
consistency                                                    slurry
otherCode                                                         NaN
material4                                                         NaN
material3                                                         NaN
material2                                                         NaN
material                                                          NaN
mType                                                             NaN
composite2                                                        NaN
composite1                                                  excavato

In [223]:
# see train data prediction
# one_row = df.iloc[row]
# print('predicted material category:')
# learn.predict(one_row)[0]

# see test data prediction
with_test_one_row = test_df.iloc[row]
learn.predict(with_test_one_row)[0]


Category ground


- tfm - transformations,
- train_ds - trained data set,
- df_tst = data fram etest,


In [0]:
# apply the set of transforms to your new data:
for tfm in data.tfms: df = tfm(df, is_test=True)

# Then you have to transforms the categorical variables into codes:
cats = np.stack([c.cat.codes.values for n,c in df[cat_names].items()], 1) + 1

# and normalize the continuous variables
conts = np.stack([c.astype('float32').values for n,c in df[cont_names].items()], 1)
means, stds = data.stats 
conts = (conts - means[None]) / (stds[None]+1e-7)

# And finally, you can run the model on [tensor(cats),tensor(conts)] (or some batches of it if you have a lot things to predict).